# IG360 QA - Create Multi-Profile
*Purpose:* Create a short profile for multiple users:
<ul>
    <li>User name, full name</li>
    <li>Stats on followers, following</li>
    <li>Date of most recent post</li>
</ul>
<br>
<br>
For this example we will look at users that target account follows though they do not follow back

In [ ]:
# import IG360 data
from IG360.ig360_scrape import *

# import other key libraries
import pandas as pd
import datetime
import json
import re
from pathlib import Path

# set output folder
txt_dir_output = '_output/'
dir_output = Path(txt_dir_output)

# locate configuration file
config_file_path = 'config.ini'
config = init_config(config_file_path)

# set user and password (turned off by default; using config.ini)
#user_login = 'xx'
#user_password = 'xx'

# set target account and capture parameters
#tgt_user = "ghost.acolyte.v2"
#tgt_user = "worldofbenches"
tgt_user = "bench.view"

Initialize Scraper and Login

In [ ]:
#scraper = IG360Scrape(config, user_login, user_password)
scraper = IG360Scrape(config)
scraper.login()

Get list of every account following or being followed by user

In [ ]:
# extract follower and following data
scraper.scrape_followers(tgt_user)
scraper.scrape_following(tgt_user)

# dump partitioned follow relationship data
part_users = scraper.get_user_part_lists()
followers_only = part_users[0]
following_only = part_users[1]
mutual_follow = part_users[2]

For each non-reciprocating account, combine short profile with date of most recent post

In [ ]:
profiles = []

for usr in following_only:
    # get profile info
    scraper.scrape_profile(usr)
    profile_data = scraper.get_profile_data()
    # get post data
    scraper.scrape_post_list(usr, 1)
    post_list = scraper.get_posts()
    if len(post_list) > 0:
        scraper.scrape_post(post_list[0].split('/')[-2], get_likes=False)
        post_data = scraper.get_post_data()
        # combine and append
        profiles.append(list(profile_data) + [post_data[1]])
    else:
        profiles.append(list(profile_data) + [None])


Process Final Export; Output to JSON

In [ ]:


# assemble multi-profile data
profile_data = []
for profile in profiles:
    key_fields = ['user_name', 'ind_verified', 'num_posts', 'num_followers', 'num_following', 'full_name', 'description', 'dt_last_post']
    new_dict_record = dict(zip(key_fields,profile))
    profile_data.append(new_dict_record)
dict_profiles = {
    "profiles" : profile_data
}

# create header dictionary
dt_run = datetime.datetime.now()
dict_header = {
    "run_datetime" : dt_run.strftime('%Y-%m-%dT%H:%M:%SZ'),
    "ig_record_type" : "mp",
    "code_version" : scraper.code_version
}

# combine all dictionaries
dict_final = {**dict_header, **dict_profiles}

# output as JSON
fn_out = re.sub(r'[\W]','',dict_header["ig_record_type"]) + '_' + tgt_user + '_' \
    + str(dt_run.year) + str(dt_run.month).zfill(2) + str(dt_run.day).zfill(2) + str(dt_run.hour).zfill(2) + str(dt_run.minute).zfill(2) + '.json'
fn_out_full = dir_output / fn_out
with open(fn_out_full, 'w') as json_file:
    json.dump(dict_final, json_file)
  
    
time.sleep(60)

In [ ]:
scraper.util_json_write_mp(tgt_user, dir_output, profiles)